# Generators
***
A generator in python is a function that returns a set of items one at a time. In other words, you can iterate over the functions and retrieve each object.

Generators look exactly like functions but instead of returning elements, you `yield` elements.

In [1]:
def range_gen(n=3):
    i = 0
    while i < n:
        yield i
        i += 1

print(type(range_gen()))

<class 'generator'>


In [2]:
range_5 = range_gen(5)
for i in range_5:
    print(i, end=' ')
    
print('\n')
# or just
for i in range_gen(5):
    print(i, end=' ')

0 1 2 3 4 

0 1 2 3 4 

Above we defined our first generator, `range_gen`. All it does, is feed us the numbers 0 to n, one at a time. 

Every time we call range_5 in the for loop, the generator **yields** the next value. This tells us that it maintains some sort of state remembering where it is. But why do we care? Why not just return the entire list of numbers from 0 to n and iterate over that in the for loop?

The reason is that generators are very memory efficient. Instead of creating and returning a potentially massive list of numbers (which takes up a lot of memory), the generator only has to store and give 1 number at a time.

In [3]:
from sys import getsizeof

In [4]:
def range_list(n):
    return list(range(n))

We'll compare the amount of bytes returned from the `range_list` function vs. the `range_gen` generator.

In [5]:
print(f'{getsizeof(range_list(1_000_000)):,} bytes')
print(f'{getsizeof(range_gen(1_000_000)):,} bytes')

9,000,120 bytes
128 bytes


We can see that the function takes up wayyyyy more memory than the generator does.

We don't just have to iterate over generators in a for loop. We can also call the `next` method on it to retrieve each value. But if we call `next` more times than there are values to yield than python will raise a StopIteration exception.

When we iterate over a generator using a for-loop, in the background python is really just calling `next` until it hits a StopIteration. 

In [6]:
range_5 = range_gen(5)

print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
# will raise a StopIteration exception
print(next(range_5))

0 1 2 3 4 

StopIteration: 

Once we iterate through all items in a generator object, the object is effectively "empty" so if we want to reiterate over the sequence we need to recreate the generator object.

In [7]:
# see we get no output because we already retrieved 
# all the elements available in range_5
for i in range_5:
    print(i)
    
print('======')
# if we want to start again we have to reinitialize the object
range_5 = range_gen(5)
for i in range_5:
    print(i, end=' ')

0 1 2 3 4 

Before we really move on we'll add some print statements to the generator to really see whats going on.

In [8]:
def range_gen(n=3):
    print('Starting')
    i = 0
    while i < n:
        print('Yielding ', end=' ')
        yield i
        print('Moving on to next value')
        i += 1
    print('Generator done')

In [9]:
for i in range_gen():
    print(i)
    print('======')

Starting
Yielding  0
Moving on to next value
Yielding  1
Moving on to next value
Yielding  2
Moving on to next value
Generator done


So we can see only the first time the generator is called is the start code run. Then once we yield a value the generator pauses and gives control back to the main code (in this case the for loop with print(i)). Then the next time we iterate over `range_gen`, it resumes right where it was and works until it hits the next yield.

In [10]:
# We can also use generators to resemble infinite lists.
def infinity():
    i = 0
    while True:
        yield i
        i += 1

In [11]:
for i in infinity():
    print(i, end=' ')
    
    if i > 50:
        break

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 

In [12]:
infty = infinity()

In [31]:
# run this cell as many times as you want 
# (CTRL + ENTER) to run a cell and remain at the same cell
next(infty)

18

Once we have a generator if we want the full sequence at once we can just convert it into a list using `list(gen)`. (If you try this with `infty` it won't terminate.)

In [32]:
def range_gen(n=3):
    i = 0
    while i < n:
        yield i
        i += 1

In [33]:
x = list(range_gen(10))
print(type(x))
x

<class 'list'>


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Sending values to generators

We see that generators allow us to call a (generator) function, run some computation and then pause until the next time we call it (more specifically call `next`). Wouldn't it be convenient if between calls we could dynamically send different values to participate in the generators computation. 

We can actually do this using the generator's `send` method. To capture the value we send to the generator we just change:
```python
yield expression
```
to
```python
sent_value = yield expression
```

We'll modify `infinty` to allow us to restart counting from any number we want whenever we want.

In [34]:
def infinity():
    i = 0
    while True:
        start = yield i
        if isinstance(start, int):
            i = start
            yield i
        i += 1

In [35]:
inf = infinity()

print('start counting from 0:')
print(next(inf), end='  ')
print(next(inf), end='  ')
print(next(inf), end='  ')
print(next(inf), end='  ')
print(next(inf), end='  ')
print(next(inf))

print('\nstart counting from 4:')
# we send the value 4 to the generator.
print(inf.send(4), end='  ')
print(next(inf), end='  ')
print(next(inf), end='  ')
print(next(inf), end='  ')
print(next(inf))

print('\nstart counting from 400:')
# we send the value 400 to the generator.
print(inf.send(400), end='  ')
print(next(inf), end='  ')
print(next(inf), end='  ')
print(next(inf))

start counting from 0:
0  1  2  3  4  5

start counting from 4:
4  5  6  7  8

start counting from 400:
400  401  402  403


When we send a value to the generator it resumes the code inside the generator until again the next yield statement.

# Generator expressions

Just like in lists we have list comprehensions to create a list in one line, we also have generator expressions. 

Generator expressions look exactly like list comprehensions except
1. They return a generator.
1. We surround the expression with ( ) instead of \[ \].

In [36]:
# we can define the same generator as above
gen = (i for i in range(5))
gen

<generator object <genexpr> at 0x107525ad0>

In [37]:
for i in gen:
    print(i, end=' ')

0 1 2 3 4 

Just like in list comprehensions we can even filter for values we want.

In [38]:
gen_even = (i for i in range(10) if i % 2 == 0)

# this 
while True:
    try:
        print(next(gen_even), end=' ')
    except StopIteration:
        break
        
print('\n==========')
gen_even = (i for i in range(10) if i % 2 == 0)

# is equivalent to this
for i in gen_even:
    print(i, end=' ')

0 2 4 6 8 
0 2 4 6 8 